In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )


intitate_notebook()

Current directory for notebook:  /workspace


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from src.data.explore_column_info import get_column_summary
from pyspark.sql.functions import col

pd.set_option("display.max_columns", 500)

In [3]:
spark = (
    SparkSession.builder.appName("PostgresETL")
    .config("spark.jars", "setup_files/postgresql-42.7.5.jar")
    .getOrCreate()
)

25/05/03 10:10:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/03 10:10:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
username = "data_source_user"
password = "data_source_user_password"
host = "172.17.0.1"
port = "5435"
database = "data_source_db"


jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}"
properties = {"user": username, "password": password, "driver": "org.postgresql.Driver"}

In [5]:
df = spark.read.option("failFast", "true").jdbc(
    url=jdbc_url, table="application_train", properties=properties
)
df

DataFrame[main_split_id: bigint, sk_id_curr: bigint, target: bigint, name_contract_type: string, code_gender: string, flag_own_car: string, flag_own_realty: string, cnt_children: bigint, amt_income_total: double, amt_credit: double, amt_annuity: double, amt_goods_price: double, name_type_suite: string, name_income_type: string, name_education_type: string, name_family_status: string, name_housing_type: string, region_population_relative: double, days_birth: bigint, days_employed: bigint, days_registration: double, days_id_publish: bigint, own_car_age: double, flag_mobil: bigint, flag_emp_phone: bigint, flag_work_phone: bigint, flag_cont_mobile: bigint, flag_phone: bigint, flag_email: bigint, occupation_type: string, cnt_fam_members: double, region_rating_client: bigint, region_rating_client_w_city: bigint, weekday_appr_process_start: string, hour_appr_process_start: bigint, reg_region_not_live_region: bigint, reg_region_not_work_region: bigint, live_region_not_work_region: bigint, reg_

In [6]:
# for col in df.columns:
#     get_column_summary(df=df, column_name=col)
#     print("\n\n")

In [7]:
col_index = 0
get_column_summary(df=df, column_name=df.columns[col_index])

Summary for column: main_split_id
Data Type        : LongType()
Unique Values    : 12
Null Count       : 0
Total Rows       : 307511
Null Percentage  : 0.0 %
Minimum value: 1
Maximum value: 12


Top 5 unique values : [7, 10, 12, 8, 1]


In [ ]:
col_index = 1
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
col_index = 2
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
col_index = 3
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
# from pyspark.ml.feature import StringIndexer

# indexer = StringIndexer(
#     inputCol="name_contract_type", outputCol="name_contract_type_string_indexed"
# )
# model = indexer.fit(df)
# df = model.transform(df)

# from pyspark.sql import Row

# temp_df = spark.createDataFrame(
#     [Row(name_contract_type=item) for item in model.labels]
# )
# temp_df.show()
# yo_indexer = StringIndexer(
#     inputCol="name_contract_type", outputCol="name_contract_type_Index"
# )

# yo_model = yo_indexer.fit(temp_df)
# yo_model.labels == model.labels

In [ ]:
col_index = 4
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
col_index = 5
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
col_index = 6
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
train, testing

In [ ]:
for col_name in [
    # "name_contract_type",
    # "code_gender",
    "flag_own_car",
    "flag_own_realty",
]:
    indexer = StringIndexer(inputCol=col_name, outputCol=col_name + "_string_indexed")
    model = indexer.fit(df)
    df = model.transform(df)

In [ ]:
col_index = 7
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
col_index = 8
get_column_summary(df=df, column_name=df.columns[col_index])

In [ ]:
col_index = 9
get_column_summary(df=df, column_name=df.columns[col_index])

In [8]:
col_index = 10
get_column_summary(df=df, column_name=df.columns[col_index])

Summary for column: amt_annuity
Data Type        : DoubleType()
Unique Values    : 13672
Null Count       : 12
Total Rows       : 307511
Null Percentage  : 0.003902299429939092 %
Minimum value: 1615.5
Maximum value: 258025.5


Top 5 unique values : [9000.0, 13500.0, 6750.0, 10125.0, 37800.0]


In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["amt_credit"], outputCol="amt_annuity_lr_features"
)

df_assembled = assembler.transform(df.filter(col("amt_annuity").isNotNull()))

lr = LinearRegression(
    featuresCol="amt_annuity_lr_features",
    labelCol="amt_annuity",
    regParam=0.3,
    elasticNetParam=0.8,
)

lrModel = lr.fit(df_assembled)

print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
print("RMSE: %f" % lrModel.summary.rootMeanSquaredError)
print("R2 Score: %f" % lrModel.summary.r2)

In [ ]:
from pyspark.sql.functions import expr, when

coefficient = lrModel.coefficients[0]
intercept = lrModel.intercept
expr_string = f"({coefficient} * amt_credit) + {intercept}"

df = df.withColumn(
    "amt_annuity",
    when(col("amt_annuity").isNull(), expr(expr_string)).otherwise(col("amt_annuity")),
)

In [9]:
col_index = 11
get_column_summary(df=df, column_name=df.columns[col_index])

Summary for column: amt_goods_price
Data Type        : DoubleType()
Unique Values    : 1002
Null Count       : 278
Total Rows       : 307511
Null Percentage  : 0.09040327012692229 %
Minimum value: 40500.0
Maximum value: 4050000.0


Top 5 unique values : [450000.0, 225000.0, 675000.0, 900000.0, 270000.0]


In [ ]:
assembler = VectorAssembler(
    inputCols=["amt_credit"], outputCol="amt_goods_price_lr_features"
)

df_assembled = assembler.transform(df.filter(col("amt_goods_price").isNotNull()))

lr = LinearRegression(
    featuresCol="amt_goods_price_lr_features",
    labelCol="amt_goods_price",
    regParam=0.3,
    elasticNetParam=0.8,
)

lrModel = lr.fit(df_assembled)

print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
print("RMSE: %f" % lrModel.summary.rootMeanSquaredError)
print("R2 Score: %f" % lrModel.summary.r2)

In [ ]:
from pyspark.sql.functions import expr, when

coefficient = lrModel.coefficients[0]
intercept = lrModel.intercept
expr_string = f"({coefficient} * amt_credit) + {intercept}"

df = df.withColumn(
    "amt_goods_price",
    when(col("amt_goods_price").isNull(), expr(expr_string)).otherwise(col("amt_goods_price")),
)

In [ ]:
col_index = 12
get_column_summary(df=df, column_name=df.columns[col_index])